# アンサンブル学習とランダムフォレスト

一群の予測機(分析器や回帰器)の予測を一つにまとめる、もっと優れている予測器の答えの方が良い予測を得られることが多い、この予測器のグループを`アンサンブル`と呼ぶ。この手法を`アンサンブル学習`、アンサンブル学習アルゴリズムを`アンサンブルメソッド`と呼ぶ

例として、訓練セットから無作為に作った様々サブセットを使って一連の決定木分類器を訓練し、予測するときには全ての決定木の予測を集め、多数決で予測クラスを決める。このような決定木のアンサンブルを`ランダムフォレスト`と呼ぶ。

In [3]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [11]:
log_clf = LogisticRegression(solver="liblinear", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=10, random_state=42)
svm_clf = SVC(gamma="auto", random_state=42)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=42, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)), ('rf', Rando...f',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False))],
         flatten_transform=None, n_jobs=None, voting='hard', weights=None)

In [12]:
from sklearn.metrics import accuracy_score

In [13]:
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.872
SVC 0.888
VotingClassifier 0.896


投票分類器は個別の分類器のどれよりもわずかながら高い性能を示している。

全ての分類器がクラスに属する確率を推計できるメソッド(`predict_proba()`)がある場合、個別の分類器の確率を平均し、もっとも確率が高いクラスを返すことができるようにする。これを`ソフト投票`と呼ぶ。この方法だと、自身の高い投票の重みがますため、ハード投票よりも高い性能を示すことが多い。

In [18]:
# votingをsoftにする
# SVCのprobabilityハイパーパラメータをTrueにする
log_clf = LogisticRegression(solver="liblinear", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=10, random_state=42)
svm_clf = SVC(gamma="auto", probability=True, random_state=42)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='soft')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=42, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)), ('rf', Rando...bf',
  max_iter=-1, probability=True, random_state=42, shrinking=True,
  tol=0.001, verbose=False))],
         flatten_transform=None, n_jobs=None, voting='soft', weights=None)

In [19]:
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.872
SVC 0.888
VotingClassifier 0.912


ハード投票よりも高い性能が出た

## バギングとペースティング
訓練セットから無作為に別々のサブセットをサンプリングして訓練するというアプローチもある。サンプリングが重複でありで行われるときは`バギング`、重複なしで行われるときは`ペースティング`と呼ぶ。

全ての分類器を予測したら、アンサンブルは単純に全ての予測器の予測を集計して新しいインスタンスに対する予測をする。  
集計関数は、一般的に分類では`モード`(ハード投票と同じく予測の最頻値をとる)、回帰は平均。

### scikit-learnにおけるバギングとペースティング

In [20]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier